In [141]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from datetime import datetime, timedelta

In [4]:
crime_df = pd.read_csv('data/Crimes_-_2001_to_2021.csv', parse_dates=['Date'], infer_datetime_format=True)
crime_df.drop(columns=[ 'Case Number','Block','IUCR','Description','Arrest',
                        'Domestic','Beat','District','Ward','Community Area',
                        'FBI Code','Updated On','Location','Year'], inplace=True)
crime_df = crime_df.rename(columns={'X Coordinate':'x', 'Y Coordinate':'y'})
crime_df.dropna(inplace=True)
crime_df.drop(crime_df[crime_df['x'] == 0].index, inplace=True)
crime_df['Date'] = crime_df['Date'].dt.date

In [5]:
# Create grid cells based on x and y coordinates
path = 'data/shapefile/geo_export.shp'
chicago = gpd.read_file(path)
chicago = chicago.dissolve()
xmin, ymin, xmax, ymax = chicago.total_bounds
n_x_cells = 50
x_cell_size = (xmax - xmin) / n_x_cells
n_y_cells = round(((xmax - xmin)/(ymax - ymin))*n_x_cells)
y_cell_size = (ymax - ymin) / n_y_cells

# Determine grid cell coordinate based on x and y coordinates
crime_df['x_cell'] = ((crime_df['Longitude'] - xmin) // x_cell_size).astype(int)
crime_df['y_cell'] = ((crime_df['Latitude'] - ymin) // y_cell_size).astype(int)
crime_df['x_y_cell'] = list(zip(crime_df['x_cell'], crime_df['y_cell']))

In [6]:
display(crime_df.head())

,ID,Date,Primary Type,Location Description,x,y,Latitude,Longitude,x_cell,y_cell,x_y_cell
0,10224738,2015-09-05,BATTERY,RESIDENCE,1165074.0,1875917.0,41.815117,-87.670000,32,24,"(32, 24)"
1,10224739,2015-09-04,THEFT,CTA BUS,1138875.0,1904869.0,41.895080,-87.765400,21,36,"(21, 36)"
3,10224740,2015-09-05,NARCOTICS,SIDEWALK,1152037.0,1920384.0,41.937406,-87.716650,26,42,"(26, 42)"
4,10224741,2015-09-05,ASSAULT,APARTMENT,1141706.0,1900086.0,41.881903,-87.755121,22,34,"(22, 34)"
5,10224742,2015-09-05,BURGLARY,RESIDENCE,1168430.0,1850165.0,41.744379,-87.658431,33,14,"(33, 14)"


In [33]:
print(crime_df['Primary Type'].unique())

soft_list = [   
    'DECEPTIVE PRACTICE', 'INTERFERENCE WITH PUBLIC OFFICER', 'GAMBLING', 'LIQUOR LAW VIOLATION',
    'NON - CRIMINAL', 'NON-CRIMINAL', 'NON-CRIMINAL (SUBJECT SPECIFIED)', 'PUBLIC PEACE VIOLATION'
    ]
soft_list = list(zip(soft_list, [0]*len(soft_list)))

middle_list = [
    'INTIMIDATION', 'PROSTITUTION', 'OTHER OFFENSE', 'CRIMINAL DAMAGE', 'STALKING', 'RITUALISM',
    'CONCEALED CARRY LICENSE VIOLATION', 'PUBLIC INDECENCY', 'NARCOTICS', 'CRIMINAL TRESPASS', 'OTHER NARCOTIC VIOLATION'
    ]
middle_list = list(zip(middle_list, [0.5]*len(middle_list)))

bad_list = [
    'BATTERY', 'THEFT', 'ASSAULT', 'BURGLARY', 'ROBBERY', 'WEAPONS VIOLATION', 'MOTOR VEHICLE THEFT', 'SEX OFFENSE', 
    'OFFENSE INVOLVING CHILDREN', 'CRIM SEXUAL ASSAULT','CRIMINAL SEXUAL ASSAULT', 'HUMAN TRAFFICKING', 'OBSCENITY',
    'ARSON', 'KIDNAPPING', 'HOMICIDE', 'DOMESTIC VIOLENCE'
    ]
bad_list = list(zip(bad_list, [1]*len(bad_list)))

score_dict = dict(soft_list+middle_list+bad_list)
test_crime_df = crime_df.replace({"Primary Type": score_dict})

In [58]:
# Group crime data by date and grid cell and create a new column for the number of crimes
grouped_crimes_df = test_crime_df.groupby(['Date','x_y_cell']).agg(
    {   
        'Primary Type': 'mean',
        'ID':'count'
    }).reset_index()
grouped_crimes_df = grouped_crimes_df.rename(columns={'ID':'Count'})

grouped_crimes_df[['x_cell', 'y_cell']] = grouped_crimes_df['x_y_cell'].to_list()

display(grouped_crimes_df)

,Date,x_y_cell,Primary Type,Count,x_cell,y_cell
0,2001-01-01,"(4, 48)",0.666667,9,4,48
1,2001-01-01,"(11, 47)",1.000000,2,11,47
2,2001-01-01,"(11, 48)",0.500000,1,11,48
3,2001-01-01,"(12, 48)",0.500000,2,12,48
4,2001-01-01,"(13, 43)",0.500000,1,13,43
...,...,...,...,...,...,...
3758299,2021-12-31,"(46, 13)",1.000000,1,46,13
3758300,2021-12-31,"(46, 15)",1.000000,2,46,15
3758301,2021-12-31,"(46, 16)",0.500000,1,46,16
3758302,2021-12-31,"(47, 1)",0.000000,1,47,1


In [48]:
# Check for missing days
deltas = grouped_crimes_df['Date'].diff()[1:]
gaps = deltas[deltas > timedelta(days=1)]
print(f'{len(gaps)} gaps with average gap duration: {gaps.mean()}')

0 gaps with average gap duration: NaT


Shape input matrix: (1, 365*20, 55, 50) voor tijdens het testen gebruiken we alleen Count als feature

In [142]:
day_group_crimes_df = grouped_crimes_df.groupby('Date')
all_days = []
for day_df in day_group_crimes_df:
    day_array = np.zeros((n_y_cells, n_x_cells))
    for index, row in day_df[1].iterrows():
        day_array[row['y_cell'], row['x_cell']] = row['Count']
    all_days.append(list(day_array))
all_days = np.array(all_days)
np.save('data/X_crimes_only.npy', all_days)

In [143]:
print(all_days.shape)

(7670, 55, 50)
